In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                    options=[
                                        {'label': 'All Sites', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'site1'},
                                        {'label': 'KSC LC-39A', 'value': 'site2'},
                                        {'label': 'VAFB SLC-4E', 'value': 'site3'},
                                        {'label': 'CCAFS SLC-40', 'value': 'site4'},
                                    ],
                                    value='ALL',
                                    placeholder="Select a launch site",
                                    searchable=True
                                    ),

                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                 dcc.RangeSlider(id='payload-slider',
                                    min=0, max=10000, step=1000,
                                    marks={0: '0', 1000: '1000', 2000: '2000',
                                            3000: '3000', 4000: '4000', 5000: '5000',
                                            6000: '6000', 7000: '7000', 8000: '8000', 
                                            9000: '9000', 10000: '10000'},
                                    value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    site_nums = ['site1', 'site2', 'site3', 'site4']
    site_names = ['CCAFS LC-40', 'KSC LC-39A', 'VAFB SLC-4E', 'CCAFS SLC-40']

    if entered_site == 'ALL':    
        fig = px.pie(spacex_df, values='class', names='Launch Site', title='All sites')
        return fig
    else:
        for site_num, site_name in zip(site_nums, site_names):
            if entered_site == site_num:
                filtered_df = spacex_df[spacex_df['Launch Site']==site_name]

                fig = px.pie(filtered_df, values='class', names='Launch Site', title=site_name)
                return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value'))
def get_scatter_plot(entered_site, entered_payload):
    site_nums = ['site1', 'site2', 'site3', 'site4']
    site_names = ['CCAFS LC-40', 'KSC LC-39A', 'VAFB SLC-4E', 'CCAFS SLC-40']
   
    payload_df = spacex_df[spacex_df['Payload']==entered_payload]
   
    if entered_site == 'ALL':  
        fig = px.scatter(payload_df, x='Payload Mass (kg)', y='payload_df', color='Booster Version Category')
        return fig
    else:
        for site_num, site_name in zip(site_nums, site_names):
            if entered_site == site_num:
                final_df = payload_df[payload_df['Launch Site']==site_name]

                fig = px.scatter(final_df, x='Payload Mass (kg)', y='payload_df', color='Booster Version Category')
                return fig


# Run the app
if __name__ == '__main__':
    app.run_server()
